**File Path**
- dbfs:/FileStore/tables/Diagnosis_details-1.csv
- dbfs:/FileStore/tables/Emergency_details-1.csv
- dbfs:/FileStore/tables/Health_details-1.csv
- dbfs:/FileStore/tables/Medical_details-1.csv
- dbfs:/FileStore/tables/Member_details-1.csv
- dbfs:/FileStore/tables/Procedure_details-1.csv


In [0]:
#Importing all the necessary packages
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [0]:
#Reading diagnosis_details file
diagnosis_df = spark.read.format('csv')\
                            .option('inferschema',True)\
                            .option('header',True)\
                            .load('dbfs:/FileStore/tables/Diagnosis_details-1.csv')

In [0]:
display(diagnosis_df.limit(10))

Diagnosis_code,Diagnosis_description
N4601,Organic azoospermia
V828,Occupant of streetcar injured in other specified transport accidents
S63211D,"Subluxation of metacarpophalangeal joint of left index finger, subsequent encounter"
V0199,"Pedestrian with other conveyance injured in collision with pedal cycle, unspecified whether traffic or nontraffic accident"
S04039A,"Injury of optic tract and pathways, unspecified eye, initial encounter"
N602,Fibroadenosis of breast
S7811,null
M62159,"Other rupture of muscle (nontraumatic), unspecified thigh"
N99518,Other cystostomy complication
S43139,"Dislocation of unspecified acromioclavicular joint, greater than 200% displacement"


In [0]:
#removing extra spaces using Trim function
diagnosis_df = diagnosis_df.select([trim(col(c)).alias(c) for c in diagnosis_df.columns])

In [0]:
# checking whether Diagnosis_code contains duplicate value or not
diagnosis_df.groupBy('Diagnosis_code').count().orderBy('count',ascending = False)

Out[102]: DataFrame[Diagnosis_code: string, count: bigint]

In [0]:
from pyspark.sql import Window
# Define a window specification partitioned by Diagnosis_code and ordered by Diagnosis_description
# Use asc_nulls_last() to prioritize non-null values
window_spec = Window.partitionBy("Diagnosis_code").orderBy(col("Diagnosis_description").asc_nulls_last())

# Add a row number column to each row within the window
diagnosis_df_with_row_num = diagnosis_df.withColumn("row_num", row_number().over(window_spec))

# Filter to keep only the first row for each Diagnosis_code (non-null description takes priority)
diagnosis_df = diagnosis_df_with_row_num.filter(col("row_num") == 1).drop('row_num')

# Show the resulting DataFrame
display(diagnosis_df.limit(10))

Diagnosis_code,Diagnosis_description
A1831,Tuberculous peritonitis
A378,Whooping cough due to other Bordetella species
A421,Abdominal actinomycosis
A5481,Gonococcal meningitis
A5602,Chlamydial vulvovaginitis
B0059,Other herpesviral disease of eye
B087,Yatapoxvirus infections
B189,"Chronic viral hepatitis, unspecified"
B2790,"Infectious mononucleosis, unspecified without complication"
B3784,Candidal otitis externa


### Emergency DF

In [0]:
emergency_df = spark.read.format('csv')\
                       .option('inferschema',True)\
                        .option('header',True)\
                        .load('dbfs:/FileStore/tables/Emergency_details-1.csv')

In [0]:
display(emergency_df.limit(10))

Member_id,emergency_contact_name,emergency_contact_phone
MiMi811108442,Millicent Sainte Paul,463-567-7558
TeNe599223180,Neila Bartoszinski,728-430-2481
MiBu217454145,Budd McKim,716-431-5374
MiMe455261897,Merilyn Boulder,137-917-0326
TeSu198272777,Sula Rackam,937-890-0233
GoCa876258307,Casey Gowans,763-436-5204
GoCa876258307,Casey Gowans,763-436-5204
MiJa710166731,Jayme Trude,278-467-4986
GoBr622338922,Brittni De Biasio,752-290-7933
ApAm559755808,Amelia Malone,331-488-0701


In [0]:
emergency_df.groupBy('Member_id').count().orderBy('count',ascending= False)

Out[106]: DataFrame[Member_id: string, count: bigint]

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

# Define window specification
window_spec = Window.partitionBy('Member_id').orderBy(
    col('emergency_contact_name').asc_nulls_last(),
)

# Assign row number
assign_row = emergency_df.withColumn('row_col', row_number().over(window_spec))

# Filter to keep only the first row per partition
df_filter = assign_row.filter(col('row_col') == 1).drop('row_col')

# Display the result
emergency_df = df_filter

display(emergency_df.limit(10))


Member_id,emergency_contact_name,emergency_contact_phone
ApAb315922251,Abelard Yegorkin,354-913-8111
ApAb504391274,Abigale Popland,141-136-0623
ApAd639216428,Adolf Gyver,529-707-4967
ApAg300549833,Aggi Shoebotham,400-237-8367
ApAj677713792,Ajay Feares,602-196-2839
ApAl211380839,Alon Thornally,605-664-0522
ApAl379415659,Alena Brason,332-389-1962
ApAl426267215,Alfreda Rolf,792-183-8502
ApAl548028460,Allys Dri,177-841-6467
ApAl893411558,Alistair Plampeyn,402-701-8253


# **Health_df**

In [0]:
health_df = spark.read.format('csv')\
                       .option('inferschema',True)\
                        .option('header',True)\
                        .load('dbfs:/FileStore/tables/Health_details-1.csv')

In [0]:
display(health_df.limit(10))

Member_id,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate
MiMi811108442,Extended 24/7 success,Fully-configurable high-level data-warehouse,Inverse executive info-mediaries,AB,78.74,2.02,117,191
TeNe599223180,Right-sized impactful capacity,Seamless human-resource process improvement,Cross-group mobile middleware,O,55.85,414.1,191,40
MiBu217454145,Mandatory bottom-line workforce,Versatile user-facing monitoring,Customizable heuristic database,AB,85.67,62.23,147,109
MiMe455261897,Universal maximized internet solution,Implemented 6th generation superstructure,Monitored demand-driven database,A,21.95,230.68,186,144
TeSu198272777,Robust zero administration application,Quality-focused solution-oriented model,Operative system-worthy architecture,AB,32.17,360.92,92,184
GoCa876258307,Networked motivating process improvement,Organic real-time hierarchy,Horizontal zero tolerance system engine,B,99.73,192.45,197,179
GoCa876258307,Networked motivating process improvement,Organic real-time hierarchy,Horizontal zero tolerance system engine,B,99.73,192.45,197,179
MiJa710166731,Public-key didactic neural-net,Programmable interactive conglomeration,Down-sized tertiary project,A,13.62,417.41,57,166
GoBr622338922,Managed stable database,Managed maximized interface,Grass-roots homogeneous contingency,AB,69.3,71.82,183,48
ApAm559755808,Networked scalable workforce,Switchable fresh-thinking encryption,Inverse interactive hardware,AB,61.65,438.59,106,197


In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('Member_id').orderBy(col('medical_condition').asc_nulls_last())

assign_row = health_df.withColumn('row_col',row_number().over(window_spec))
df_filter = assign_row.filter(col('row_col') ==1).drop('row_col')
health_df = df_filter

display(health_df.limit(10))

Member_id,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate
ApAb315922251,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118
ApAb504391274,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138
ApAd639216428,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105
ApAg300549833,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199
ApAj677713792,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175
ApAl211380839,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144
ApAl379415659,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140
ApAl426267215,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181
ApAl548028460,Sharable client-driven encoding,Function-based intangible frame,Organized bandwidth-monitored Graphic Interface,B,22.47,285.35,142,55
ApAl893411558,Organic asynchronous challenge,Adaptive fault-tolerant local area network,Self-enabling methodical portal,O,44.53,213.04,56,188


### Medical DF

In [0]:
medical_df = spark.read.format('csv')\
                       .option('inferschema',True)\
                        .option('header',True)\
                        .load('dbfs:/FileStore/tables/Medical_details-1.csv')

In [0]:
display(medical_df.limit(10))

Member_id,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date
MiMi811108442,Leannon-Morissette,103082,11/20/2046,S59099P,041F09K,Millicent Sainte Paul,2/6/2016
TeNe599223180,Zemlak and Sons,103675,8/15/2042,M84539P,09R907Z,Neila Bartoszinski,10/30/2014
MiBu217454145,"Glover, Brakus and Sawayn",105096,5/2/2026,Y3702,0PUR07Z,Budd McKim,6/6/2022
MiMe455261897,"Kub, Roob and Stokes",105710,1/23/2031,V159XXS,0Y0D3KZ,Merilyn Boulder,5/31/2005
TeSu198272777,Borer Group,105847,3/29/2033,T8620,0SP034Z,Sula Rackam,11/5/2004
GoCa876258307,"Deckow, Olson and Koss",106641,9/24/2043,T445X1D,0BV10DZ,Casey Gowans,9/10/2005
GoCa876258307,"Deckow, Olson and Koss",106641,9/24/2043,T445X1D,0BV10DZ,Casey Gowans,9/10/2005
MiJa710166731,Doyle-Torp,106983,5/28/2027,S34113D,04RH47Z,Jayme Trude,12/6/2018
GoBr622338922,Batz and Sons,107191,11/19/2023,S1120,0F993ZZ,Brittni De Biasio,10/10/2012
ApAm559755808,"Buckridge, Bernier and Corkery",107766,5/27/2047,S83211S,0R9S3ZX,Amelia Malone,3/4/2016


In [0]:
medical_df = medical_df.select([trim(col(c)).alias(c) for c in medical_df.columns])

In [0]:
medical_df = medical_df.withColumn("format_date",to_date(col('last_visit_date'),"M/d/yyyy"))

In [0]:
medical_df = medical_df.filter(col("format_date").isNotNull()).drop('format_date')

In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('Member_id').orderBy(
    col('last_visit_date').asc_nulls_last()
)
assign_row = medical_df.withColumn("row_col",row_number().over(window_spec))
df_filter = assign_row.filter(col('row_col')==1).drop('row_col')

medical_df = df_filter
display(medical_df.limit(10))

Member_id,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date
ApAb315922251,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006
ApAb504391274,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005
ApAd639216428,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015
ApAg300549833,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005
ApAj677713792,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000
ApAl211380839,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014
ApAl379415659,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020
ApAl426267215,Kovacek-McLaughlin,901932,4/7/2038,A5481,00P003Z,Alfreda Rolf,8/5/2022
ApAl548028460,Friesen Inc,748725,12/30/2035,H21553,B039ZZZ,Allys Dri,4/23/2007
ApAl893411558,Skiles-Sanford,297096,5/18/2046,C84A5,06SQ0ZZ,Alistair Plampeyn,11/23/2013


### Members DF


In [0]:
members_df = spark.read.format('csv')\
                       .option('inferschema',True)\
                        .option('header',True)\
                        .load('dbfs:/FileStore/tables/Member_details-1.csv')

In [0]:
display(members_df.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company
MiMi811108442,Millicent,Sainte Paul,8/20/1984,Female,msainte7i@bloglovin.com,459-468-1581,347 Boyd Center,Borås,Microsoft
TeNe599223180,Neila,Bartoszinski,3/28/1947,Female,nbartoszinskil6@craigslist.org,822-425-0441,3026 Talisman Point,Sidi Redouane,Tesla
MiBu217454145,Budd,McKim,3/1/1956,Male,bmckimby@bigcartel.com,426-230-2515,92783 Arkansas Alley,Ribeira,Microsoft
MiMe455261897,Merilyn,Boulder,7/30/1912,Polygender,mboulder1f@123-reg.co.uk,890-808-9335,7829 Sheridan Plaza,Oetuke,Microsoft
TeSu198272777,Sula,Rackam,3/1/1964,Female,srackammz@seesaa.net,504-375-0134,61525 Dovetail Point,Colima,Tesla
GoCa876258307,Casey,Gowans,3/15/1926,Female,cgowansj9@fc2.com,949-609-8257,5 Pennsylvania Drive,Prabuty,Google
GoCa876258307,Casey,Gowans,3/15/1926,Female,cgowansj9@fc2.com,949-609-8257,5 Pennsylvania Drive,Prabuty,Google
MiJa710166731,Jayme,Trude,5/2/1920,Female,jtrudekv@wp.com,323-690-9320,656 Twin Pines Place,Los Angeles,Microsoft
GoBr622338922,Brittni,De Biasio,7/8/1946,Genderqueer,bdehm@economist.com,539-610-1845,600 North Street,Wangmo,Google
ApAm559755808,Amelia,Malone,4/16/1981,Female,amaloneo3@zdnet.com,910-297-8112,30 Melrose Court,Várzea Grande,Apple


In [0]:
#Trim is used to remove extra spaces used in df
members_df = members_df.select([trim(col(c)) .alias(c)for c in members_df.columns])

In [0]:
#Checking Hardcoded null in df
display(members_df.select([sum((col(c) == 'null').cast('int')) for c in members_df.columns]))

sum(CAST((Member_id = null) AS INT)),sum(CAST((first_name = null) AS INT)),sum(CAST((last_name = null) AS INT)),sum(CAST((date_of_birth = null) AS INT)),sum(CAST((gender = null) AS INT)),sum(CAST((email = null) AS INT)),sum(CAST((phone_number = null) AS INT)),sum(CAST((address = null) AS INT)),sum(CAST((city = null) AS INT)),sum(CAST((Company = null) AS INT))
0,0,4,0,0,33,33,0,33,0


In [0]:
members_df = members_df.withColumn("format_date",to_date(col('date_of_birth'),"M/d/yyyy"))

In [0]:
members_df = members_df.filter(col('format_date').isNotNull()).drop('format_date')

In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('Member_id').orderBy(
    col('date_of_birth').asc_nulls_last()
)

assign_row = members_df.withColumn('row_col',row_number().over(window_spec))
filter_member_df = assign_row.filter(col('row_col') == 1).drop('row_col')
#assign_row.display()
members_df = filter_member_df
display(members_df.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple
ApAl548028460,Allys,Dri,2/15/1981,Female,adril9@webs.com,557-232-1602,3988 Caliangt Court,Djohong,Apple
ApAl893411558,Alistair,Plampeyn,5/19/1930,Male,aplampeynai@arizona.edu,597-195-1810,1786 Northfield Place,Végueta,Apple


### Procedure DF

In [0]:
procedure_df = spark.read.format('csv')\
                            .option('inferschema',True)\
                            .option('header',True)\
                            .load('dbfs:/FileStore/tables/Procedure_details-1.csv/')

In [0]:
display(procedure_df.limit(10))

Procedure_code,Procedure_description
06S84ZZ,"Reposition Portal Vein, Percutaneous Endoscopic Approach"
B217YZZ,Fluoroscopy of Right Internal Mammary Bypass Graft using Other Contrast
0RHT48Z,"Insertion of Spacer into Left Metacarpocarpal Joint, Percutaneous Endoscopic Approach"
041N0JL,null
F003GKZ,Communicative/Cognitive Integration Skills Assessment of Neurological System - Whole Body using Audiovisual Equipment
0C973ZZ,"Drainage of Tongue, Percutaneous Approach"
0QWB35Z,"Revision of External Fixation Device in Right Lower Femur, Percutaneous Approach"
0DHA03Z,"Insertion of Infusion Device into Jejunum, Open Approach"
3E0CXKZ,"Introduction of Other Diagnostic Substance into Eye, External Approach"
0RW13KZ,"Revision of Nonautologous Tissue Substitute in Cervical Vertebral Joint, Percutaneous Approach"


In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('Procedure_code').orderBy(
    col('Procedure_description').asc_nulls_last(),
)
assign_row = procedure_df.withColumn('row_col',row_number().over(window_spec))
filter_procedure_df = assign_row.filter(col('row_col') == 1).drop('row_col')

procedure_df = filter_procedure_df

### Joining Tables

In [0]:
# joining Member_df and Health_df
members_health = members_df.join(health_df, members_df.Member_id == health_df.Member_id, 'inner')
display(members_health.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,Member_id,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,ApAb315922251,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,ApAb504391274,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,ApAd639216428,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,ApAg300549833,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,ApAj677713792,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,ApAl211380839,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,ApAl379415659,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,ApAl426267215,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181
ApAl548028460,Allys,Dri,2/15/1981,Female,adril9@webs.com,557-232-1602,3988 Caliangt Court,Djohong,Apple,ApAl548028460,Sharable client-driven encoding,Function-based intangible frame,Organized bandwidth-monitored Graphic Interface,B,22.47,285.35,142,55
ApAl893411558,Alistair,Plampeyn,5/19/1930,Male,aplampeynai@arizona.edu,597-195-1810,1786 Northfield Place,Végueta,Apple,ApAl893411558,Organic asynchronous challenge,Adaptive fault-tolerant local area network,Self-enabling methodical portal,O,44.53,213.04,56,188


In [0]:
members_health = members_health.drop(health_df.Member_id)

In [0]:
# joining members_health with medical_df
members_health_emergency = members_health.join(emergency_df, members_health.Member_id == emergency_df.Member_id,'inner')
display(members_health_emergency.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,Member_id,emergency_contact_name,emergency_contact_phone
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,ApAb315922251,Abelard Yegorkin,354-913-8111
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,ApAb504391274,Abigale Popland,141-136-0623
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,ApAd639216428,Adolf Gyver,529-707-4967
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,ApAg300549833,Aggi Shoebotham,400-237-8367
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,ApAj677713792,Ajay Feares,602-196-2839
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,ApAl211380839,Alon Thornally,605-664-0522
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,ApAl379415659,Alena Brason,332-389-1962
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181,ApAl426267215,Alfreda Rolf,792-183-8502
ApAl548028460,Allys,Dri,2/15/1981,Female,adril9@webs.com,557-232-1602,3988 Caliangt Court,Djohong,Apple,Sharable client-driven encoding,Function-based intangible frame,Organized bandwidth-monitored Graphic Interface,B,22.47,285.35,142,55,ApAl548028460,Allys Dri,177-841-6467
ApAl893411558,Alistair,Plampeyn,5/19/1930,Male,aplampeynai@arizona.edu,597-195-1810,1786 Northfield Place,Végueta,Apple,Organic asynchronous challenge,Adaptive fault-tolerant local area network,Self-enabling methodical portal,O,44.53,213.04,56,188,ApAl893411558,Alistair Plampeyn,402-701-8253


In [0]:
members_health_emergency = members_health_emergency.drop(emergency_df.Member_id)

In [0]:
members_health_emergency_medical = members_health_emergency.join(medical_df, members_health_emergency.Member_id == medical_df.Member_id,'inner')
display(members_health_emergency_medical.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,Member_id,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,Abelard Yegorkin,354-913-8111,ApAb315922251,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,Abigale Popland,141-136-0623,ApAb504391274,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,Adolf Gyver,529-707-4967,ApAd639216428,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,Aggi Shoebotham,400-237-8367,ApAg300549833,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,Ajay Feares,602-196-2839,ApAj677713792,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,Alon Thornally,605-664-0522,ApAl211380839,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,Alena Brason,332-389-1962,ApAl379415659,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181,Alfreda Rolf,792-183-8502,ApAl426267215,Kovacek-McLaughlin,901932,4/7/2038,A5481,00P003Z,Alfreda Rolf,8/5/2022
ApAl548028460,Allys,Dri,2/15/1981,Female,adril9@webs.com,557-232-1602,3988 Caliangt Court,Djohong,Apple,Sharable client-driven encoding,Function-based intangible frame,Organized bandwidth-monitored Graphic Interface,B,22.47,285.35,142,55,Allys Dri,177-841-6467,ApAl548028460,Friesen Inc,748725,12/30/2035,H21553,B039ZZZ,Allys Dri,4/23/2007
ApAl893411558,Alistair,Plampeyn,5/19/1930,Male,aplampeynai@arizona.edu,597-195-1810,1786 Northfield Place,Végueta,Apple,Organic asynchronous challenge,Adaptive fault-tolerant local area network,Self-enabling methodical portal,O,44.53,213.04,56,188,Alistair Plampeyn,402-701-8253,ApAl893411558,Skiles-Sanford,297096,5/18/2046,C84A5,06SQ0ZZ,Alistair Plampeyn,11/23/2013


In [0]:
members_health_emergency_medical = members_health_emergency_medical.drop(medical_df.Member_id)

In [0]:
members_health_emergency_medical_diagnosis = members_health_emergency_medical.join(diagnosis_df, members_health_emergency_medical.Diagnosis_code == diagnosis_df.Diagnosis_code, 'inner')
display(members_health_emergency_medical_diagnosis.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date,Diagnosis_code,Diagnosis_description
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,Abelard Yegorkin,354-913-8111,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006,A378,Whooping cough due to other Bordetella species
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,Abigale Popland,141-136-0623,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005,S2591XD,"Laceration of unspecified blood vessel of thorax, subsequent encounter"
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,Adolf Gyver,529-707-4967,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015,Q7240,Longitudinal reduction defect of unspecified femur
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,Aggi Shoebotham,400-237-8367,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005,I70703,"Unspecified atherosclerosis of other type of bypass graft(s) of the extremities, bilateral legs"
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,Ajay Feares,602-196-2839,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000,S52692A,"Other fracture of lower end of left ulna, initial encounter for closed fracture"
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,Alon Thornally,605-664-0522,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014,T63482D,"Toxic effect of venom of other arthropod, intentional self-harm, subsequent encounter"
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,Alena Brason,332-389-1962,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020,Y0702,"Wife, perpetrator of maltreatment and neglect"
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181,Alfreda Rolf,792-183-8502,Kovacek-McLaughlin,901932,4/7/2038,A5481,00P003Z,Alfreda Rolf,8/5/2022,A5481,Gonococcal meningitis
ApAl548028460,Allys,Dri,2/15/1981,Female,adril9@webs.com,557-232-1602,3988 Caliangt Court,Djohong,Apple,Sharable client-driven encoding,Function-based intangible frame,Organized bandwidth-monitored Graphic Interface,B,22.47,285.35,142,55,Allys Dri,177-841-6467,Friesen Inc,748725,12/30/2035,H21553,B039ZZZ,Allys Dri,4/23/2007,H21553,"Recession of chamber angle, bilat

In [0]:
members_health_emergency_medical_diagnosis = members_health_emergency_medical_diagnosis.drop(diagnosis_df.Diagnosis_code)

In [0]:
final_join = members_health_emergency_medical_diagnosis.join(procedure_df, members_health_emergency_medical_diagnosis.Procedure_code == procedure_df.Procedure_code,'inner')
display(final_join.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date,Diagnosis_description,Procedure_code,Procedure_description
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,Abelard Yegorkin,354-913-8111,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006,Whooping cough due to other Bordetella species,0F150Z9,"Bypass Right Hepatic Duct to Common Bile Duct, Open Approach"
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,Abigale Popland,141-136-0623,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005,"Laceration of unspecified blood vessel of thorax, subsequent encounter",0MQ64ZZ,"Repair Left Wrist Bursa and Ligament, Percutaneous Endoscopic Approach"
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,Adolf Gyver,529-707-4967,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015,Longitudinal reduction defect of unspecified femur,0Q9L0ZZ,"Drainage of Right Tarsal, Open Approach"
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,Aggi Shoebotham,400-237-8367,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005,"Unspecified atherosclerosis of other type of bypass graft(s) of the extremities, bilateral legs",0SWL38Z,"Revision of Spacer in Left Metatarsal-Tarsal Joint, Percutaneous Approach"
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,Ajay Feares,602-196-2839,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000,"Other fracture of lower end of left ulna, initial encounter for closed fracture",03V30ZZ,"Restriction of Right Subclavian Artery, Open Approach"
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,Alon Thornally,605-664-0522,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014,"Toxic effect of venom of other arthropod, intentional self-harm, subsequent encounter",0Q9R30Z,"Drainage of Left Toe Phalanx with Drainage Device, Percutaneous Approach"
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,Alena Brason,332-389-1962,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020,"Wife, perpetrator of maltreatment and neglect",021649P,"Bypass Right Atrium to Pulmonary Trunk with Autologous Venous Tissue, Percutaneous Endoscopic Approach"
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending wor

In [0]:
final_join = final_join.drop(procedure_df.Procedure_code)

In [0]:
#Final null check after joining dataframe
display(final_join.select([sum((col(c) == 'null').cast('int')).alias(c) for c in final_join.columns]))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date,Diagnosis_description,Procedure_description
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Saving Dataframe(DF) into Table

In [0]:
display(final_join.limit(10))

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date,Diagnosis_description,Procedure_description
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,Abelard Yegorkin,354-913-8111,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006,Whooping cough due to other Bordetella species,"Bypass Right Hepatic Duct to Common Bile Duct, Open Approach"
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,Abigale Popland,141-136-0623,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005,"Laceration of unspecified blood vessel of thorax, subsequent encounter","Repair Left Wrist Bursa and Ligament, Percutaneous Endoscopic Approach"
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,Adolf Gyver,529-707-4967,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015,Longitudinal reduction defect of unspecified femur,"Drainage of Right Tarsal, Open Approach"
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,Aggi Shoebotham,400-237-8367,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005,"Unspecified atherosclerosis of other type of bypass graft(s) of the extremities, bilateral legs","Revision of Spacer in Left Metatarsal-Tarsal Joint, Percutaneous Approach"
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,Ajay Feares,602-196-2839,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000,"Other fracture of lower end of left ulna, initial encounter for closed fracture","Restriction of Right Subclavian Artery, Open Approach"
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,Alon Thornally,605-664-0522,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014,"Toxic effect of venom of other arthropod, intentional self-harm, subsequent encounter","Drainage of Left Toe Phalanx with Drainage Device, Percutaneous Approach"
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,Alena Brason,332-389-1962,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020,"Wife, perpetrator of maltreatment and neglect","Bypass Right Atrium to Pulmonary Trunk with Autologous Venous Tissue, Percutaneous Endoscopic Approach"
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181,Alfreda Rolf,792-183-8502,Kovacek-McLaughli

In [0]:
# For the first create a databse to store table
spark.sql("CREATE DATABASE IF NOT EXISTS Health_DB")

Out[139]: DataFrame[]

In [0]:
# Using recently created database
spark.sql("use health_db")

Out[140]: DataFrame[]

In [0]:
# Saving Dataframe final_join as Table into health_db
final_join.write\
    .mode('overwrite')\
    .saveAsTable('health_details')

In [0]:
%sql
Select * from health_details
limit 10;

Member_id,first_name,last_name,date_of_birth,gender,email,phone_number,address,city,Company,medical_condition,allergies,medications,blood_type,height,weight,blood_pressure,heart_rate,emergency_contact_name,emergency_contact_phone,insurance_provider,insurance_policy_number,insurance_expiration_date,Diagnosis_code,Procedure_code,primary_care_physician,last_visit_date,Procedure_description,Diagnosis_description
ApAb315922251,Abelard,Yegorkin,7/31/1939,Non-binary,ayegorkin44@redcross.org,909-489-4526,37 Delaware Center,Menongue,Apple,Managed intermediate product,Monitored non-volatile framework,Team-oriented 24/7 capacity,A,22.31,315.11,89,118,Abelard Yegorkin,354-913-8111,"Kuvalis, Gerlach and Mills",644738,6/24/2033,A378,0F150Z9,Abelard Yegorkin,11/30/2006,"Bypass Right Hepatic Duct to Common Bile Duct, Open Approach",Whooping cough due to other Bordetella species
ApAb504391274,Abigale,Popland,6/25/1977,Female,apoplandat@google.cn,490-236-7183,2 Oakridge Park,Stęszew,Apple,Virtual explicit encoding,User-centric tertiary contingency,Virtual reciprocal matrix,O,91.5,325.13,53,138,Abigale Popland,141-136-0623,"Lesch, Greenfelder and Lakin",435017,7/22/2046,S2591XD,0MQ64ZZ,Abigale Popland,3/31/2005,"Repair Left Wrist Bursa and Ligament, Percutaneous Endoscopic Approach","Laceration of unspecified blood vessel of thorax, subsequent encounter"
ApAd639216428,Adolf,Gyver,10/28/1980,Male,agyver91@sourceforge.net,635-328-9277,57082 South Circle,Chavão,Apple,Quality-focused dynamic implementation,Open-architected discrete knowledge user,Intuitive neutral superstructure,A,5.58,46.09,162,105,Adolf Gyver,529-707-4967,Osinski LLC,726995,6/30/2035,Q7240,0Q9L0ZZ,Adolf Gyver,10/28/2015,"Drainage of Right Tarsal, Open Approach",Longitudinal reduction defect of unspecified femur
ApAg300549833,Aggi,Shoebotham,11/29/1966,Female,ashoebothamfp@etsy.com,566-658-7860,4671 Hintze Terrace,Norrköping,Apple,Advanced content-based productivity,Organic static success,Function-based local application,AB,29.34,156.47,94,199,Aggi Shoebotham,400-237-8367,Schmidt-Kuphal,870897,6/19/2029,I70703,0SWL38Z,Aggi Shoebotham,4/8/2005,"Revision of Spacer in Left Metatarsal-Tarsal Joint, Percutaneous Approach","Unspecified atherosclerosis of other type of bypass graft(s) of the extremities, bilateral legs"
ApAj677713792,Ajay,Feares,1/30/1905,Female,afearesqw@bloglines.com,555-516-4418,16019 Rigney Way,Yong’an,Apple,Distributed coherent software,Reactive next generation product,Focused encompassing array,A,46.43,211.27,122,175,Ajay Feares,602-196-2839,Corwin Inc,255481,1/11/2031,S52692A,03V30ZZ,Ajay Feares,4/18/2000,"Restriction of Right Subclavian Artery, Open Approach","Other fracture of lower end of left ulna, initial encounter for closed fracture"
ApAl211380839,Alon,Thornally,1/26/1977,Male,athornallyh2@boston.com,295-927-2710,5683 Corry Circle,Abelheira,Apple,User-centric empowering forecast,Networked responsive core,Managed non-volatile matrix,A,26.85,245.72,182,144,Alon Thornally,605-664-0522,Lynch-Rice,194338,11/13/2029,T63482D,0Q9R30Z,Alon Thornally,10/18/2014,"Drainage of Left Toe Phalanx with Drainage Device, Percutaneous Approach","Toxic effect of venom of other arthropod, intentional self-harm, subsequent encounter"
ApAl379415659,Alena,Brason,1/7/1911,Female,abrasongy@amazon.co.uk,508-832-9106,25 Hayes Way,Gesikan,Apple,Function-based systemic website,Synchronised executive support,Digitized optimal website,B,85.04,76.97,146,140,Alena Brason,332-389-1962,Ritchie and Sons,164682,11/16/2024,Y0702,021649P,Alena Brason,3/18/2020,"Bypass Right Atrium to Pulmonary Trunk with Autologous Venous Tissue, Percutaneous Endoscopic Approach","Wife, perpetrator of maltreatment and neglect"
ApAl426267215,Alfreda,Rolf,12/17/1928,Female,arolf1s@parallels.com,355-242-6585,385 Di Loreto Alley,Dolní Cerekev,Apple,Persistent optimal benchmark,De-engineered stable infrastructure,Re-contextualized upward-trending workforce,O,42.81,63.68,74,181,Alfreda Rolf,792-183-8502,Kovacek-McLaughli

In [0]:
 # getting a patient data which have leukemia in diagnosis description
 reporting_table = spark.sql(""" 
SELECT
        first_name,
      last_name, 
      date_of_birth,
      gender,
      phone_number,
      address,
      height,
      weight,
      blood_pressure,
      blood_type,
      emergency_contact_name,
      emergency_contact_phone,
      last_visit_date,
      Diagnosis_description,
      primary_care_physician,
      Procedure_description
From health_details
Where Diagnosis_description LIKE "%leukemia%"
""")
reporting_table.display()

first_name,last_name,date_of_birth,gender,phone_number,address,height,weight,blood_pressure,blood_type,emergency_contact_name,emergency_contact_phone,last_visit_date,Diagnosis_description,primary_care_physician,Procedure_description
Gisella,Laydon,12/5/1926,Polygender,523-508-7703,41847 Corben Crossing,81.27,278.66,150,O,Gisella Laydon,177-401-9013,10/31/2021,Acute myelomonocytic leukemia,Gisella Laydon,"Replacement of Left Eye with Synthetic Substitute, Percutaneous Approach"
Lea,Triggol,3/30/1989,Female,287-305-0131,13 Comanche Street,80.84,76.01,200,O,Lea Triggol,605-219-9680,10/1/2008,"Chronic myelomonocytic leukemia, in remission",Lea Triggol,"Insertion of Intramedullary Internal Fixation Device into Left Femoral Shaft, Open Approach"


In [0]:
%sql
use database health_db

In [0]:
reporting_table.write\
    .option('mergeSchema',True)\
    .mode('overwrite')\
    .saveAsTable('reporting_details')

In [0]:
%sql
select * from reporting_details

first_name,last_name,date_of_birth,gender,phone_number,address,height,weight,blood_pressure,blood_type,emergency_contact_name,emergency_contact_phone,last_visit_date,Diagnosis_description,primary_care_physician,Procedure_description
Gisella,Laydon,12/5/1926,Polygender,523-508-7703,41847 Corben Crossing,81.27,278.66,150,O,Gisella Laydon,177-401-9013,10/31/2021,Acute myelomonocytic leukemia,Gisella Laydon,"Replacement of Left Eye with Synthetic Substitute, Percutaneous Approach"
Lea,Triggol,3/30/1989,Female,287-305-0131,13 Comanche Street,80.84,76.01,200,O,Lea Triggol,605-219-9680,10/1/2008,"Chronic myelomonocytic leukemia, in remission",Lea Triggol,"Insertion of Intramedullary Internal Fixation Device into Left Femoral Shaft, Open Approach"
